In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp

import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.metricBundles as metricBundles

In [2]:
dbFile = 'feature_baseline_10yrs.db'
# This connection checks which version it thinks the database is and should do the right thing schema-wise
conn = db.Database(dbFile, defaultTable='observations')
outDir='temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [4]:
mjdCol='mjd'
latCol = 'dec'
lonCol = 'RA'
expTimeCol='exptime'

bundleList = []

metric = metrics.BruteOSFMetric(expTimeCol=exptimeCol, mjdCol='mjd')
slicer = slicers.UniSlicer()
sql = ''
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

sql='dec > %f and dec < %f and ra > %f and ra < %f' (dec1, dec2, ra1, ra2)
# Let's check the median airmass and number of observations
metric = metrics.MedianMetric(col='airmass')
slicer = slicers.UniSlicer()
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

metric = metrics.CountMetric(col=mjdCol)
slicer=slicers.HealpixSlicer(latCol=latCol, lonCol=lonCol)
ss = metrics.MedianMetric()
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, summaryMetrics=ss))



In [5]:
bd = metricBundles.makeBundlesDictFromList(bundleList)
bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
bg.runAll()

Querying database observations with no constraint for columns ['exptime', 'mjd'].
Found 2304791 visits
Running:  ['opsim_BruteOSFMetric_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [8]:
print('Feature Based Open Shutter Fraction=%f' % bundleList[0].metricValues.data)
print('Median airmass = %f' % bundleList[1].metricValues.data)
print('Median Count = %f' % bundleList[2].summaryValues[0])

Feature Based Open Shutter Fraction=0.705491


In [9]:
mjdCol='expMJD'
latCol = 'fieldDec'
lonCol = 'fieldRA'
exptimeCol='expTime'

dbFile = 'minion_1016_sqlite.db'
# This connection checks which version it thinks the database is and should do the right thing schema-wise
conn = db.OpsimDatabase(dbFile)
outDir='temp'
resultsDb = db.ResultsDb(outDir=outDir)
bundleList = []

metric = metrics.BruteOSFMetric(expTimeCol=expTimeCol=exptimeCol, mjdCol=mjdCol)
slicer = slicers.UniSlicer()
sql = ''
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

sql='dec > %f and dec < %f and ra > %f and ra < %f' (dec1, dec2, ra1, ra2)
# Let's check the median airmass and number of observations
metric = metrics.MedianMetric(col='airmass')
slicer = slicers.UniSlicer()
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

metric = metrics.CountMetric(col=mjdCol)
slicer=slicers.HealpixSlicer(latCol=latCol, lonCol=lonCol)
ss = metrics.MedianMetric()
bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, summaryMetrics=ss))


bd = metricBundles.makeBundlesDictFromList(bundleList)
bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
bg.runAll()


print('Minion Open Shutter Fraction=%f' % bundleList[0].metricValues.data)

Querying database Summary with no constraint for columns ['expMJD', 'visitExpTime'].
Found 2447931 visits
Running:  ['opsim_BruteOSFMetric_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Minion Open Shutter Fraction=0.735663


In [10]:
dbFile = 'astro-lsst-01_2013.db'
# This connection checks which version it thinks the database is and should do the right thing schema-wise
conn = db.OpsimDatabase(dbFile)
outDir='temp'
resultsDb = db.ResultsDb(outDir=outDir)
bundleList = []

metric = metrics.BruteOSFMetric(expTimeCol='visitExposureTime', mjdCol='observationStartMJD')
slicer = slicers.UniSlicer()
sql = ''

bundleList.append(metricBundles.MetricBundle(metric,slicer,sql))

bd = metricBundles.makeBundlesDictFromList(bundleList)
bg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, resultsDb=resultsDb)
bg.runAll()
print('astro-lsst-01_2013 Open Shutter Fraction=%f' % bundleList[0].metricValues.data)

Querying database SummaryAllProps with no constraint for columns ['visitExposureTime', 'observationStartMJD'].
Found 2364554 visits
Running:  ['opsim_BruteOSFMetric_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Minion Open Shutter Fraction=0.715178
